In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on = 'title')

In [4]:
movies = movies[['movie_id', 'title', 'overview', 'keywords', 'genres', 'cast', 'crew']]

Data Preprocessing

In [5]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
keywords    0
genres      0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace = True)

In [7]:
movies.duplicated().sum()

0

In [8]:
import ast

In [9]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'].replace(" ", ""))
    return l

In [10]:
movies['genres'] = movies['genres'].apply(convert)

In [11]:
movies['keywords'] = movies['keywords'].apply(convert)

In [12]:
def convert3(obj):
    l = []
    count = 1
    for i in ast.literal_eval(obj):
        if count < 4:
            l.append(i['name'].replace(" ", ""))
            count += 1
        else:
            break
    return l

In [13]:
movies['cast'] = movies['cast'].apply(convert3)

In [14]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'].replace(" ", ""))
            break
    return l

In [15]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [16]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [17]:
movies['tags'] = movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']

In [18]:
new_df = movies[['movie_id', 'title', 'tags']]

In [19]:
new_df

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [20]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Dell\AppData\Local\Temp\ipykernel_7172\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [21]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\Dell\AppData\Local\Temp\ipykernel_7172\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [22]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [28]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [29]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Dell\AppData\Local\Temp\ipykernel_7172\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [30]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d action adventur fantasi sciencefict samworthington zoesaldana sigourneyweav jamescameron'

Model Building

- Vectorization

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [32]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [33]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
print(list(cv.get_feature_names_out()))

['000', '007', '10', '100', '11', '12', '13', '14', '15', '16', '17', '17th', '18', '18th', '18thcenturi', '19', '1910', '1920', '1930', '1940', '1944', '1950', '1950s', '1960', '1960s', '1970', '1970s', '1971', '1974', '1976', '1980', '1985', '1990', '1999', '19th', '19thcenturi', '20', '200', '2003', '2009', '20th', '21st', '23', '24', '25', '30', '300', '3d', '40', '50', '500', '60', '70', '80', 'aaron', 'aaroneckhart', 'abandon', 'abduct', 'abigailbreslin', 'abil', 'abl', 'aboard', 'abov', 'abu', 'academ', 'academi', 'accept', 'access', 'accid', 'accident', 'acclaim', 'accompani', 'accomplish', 'account', 'accu', 'ace', 'achiev', 'acquaint', 'act', 'action', 'actionhero', 'activ', 'activist', 'activities', 'actor', 'actress', 'actual', 'ad', 'adam', 'adamsandl', 'adamshankman', 'adapt', 'add', 'addict', 'adjust', 'admir', 'admit', 'adolesc', 'adopt', 'ador', 'adrienbrodi', 'adult', 'adultanim', 'adulteri', 'adulthood', 'advanc', 'adventur', 'adventure', 'adventures', 'adverti', 'ad

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
similarity = cosine_similarity(vectors)

- Main Function

In [37]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [38]:
recommend('Avatar')

Aliens vs Predator: Requiem
Falcon Rising
Independence Day
Titan A.E.
Aliens


In [46]:
import pickle

In [49]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl', 'wb'))

In [50]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))